## scikeras Keras regressor integration

The integration of KerasRegressor in pipeline is limited. To use full keras capabilities, you may need to modify the pipeline.

In [ ]:
# Standard loading and preprocessing code

from pinard import nirs_set as n_set
from sklearn.model_selection import train_test_split
import numpy as np

# Init basic random
rd_seed = 42
np.random.seed(rd_seed)

# Create a set named data
n = n_set.NIRS_Set('data')

# Load csv data and split into train and test
X, y = n.load('Xcal.csv', 'Ycal.csv', x_hdr=0, y_hdr=0, y_cols=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = rd_seed)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


from pinard import preprocessor as pp
from sklearn.pipeline import Pipeline

### Declare preprocessing pipeline components
preprocessing = [   ('id', pp.IdentityTransformer()),
                    ('savgol', pp.SavitzkyGolay()),
                    ('gaussian1', pp.Gaussian(order = 1, sigma = 2)),
                    ('gaussian2', pp.Gaussian(order = 2, sigma = 1)),
                    ('haar', pp.Wavelet('haar')),
                    ('savgol*savgol', Pipeline([('_sg1',pp.SavitzkyGolay()),('_sg2',pp.SavitzkyGolay())])),
                    ('gaussian1*savgol', Pipeline([('_g1',pp.Gaussian(order = 1, sigma = 2)),('_sg3',pp.SavitzkyGolay())])),
                    ('gaussian2*savgol', Pipeline([('_g2',pp.Gaussian(order = 1, sigma = 2)),('_sg4',pp.SavitzkyGolay())])),
                    ('haar*savgol', Pipeline([('_haar2',pp.Wavelet('haar')),('_sg5',pp.SavitzkyGolay())]))
                ]

## Complex pipeline

In the following example we will declare a custom callback, un custom optimizer and we will use test data as validation metrics for the keras model.

### Custom Adam optimizer with Cyclic Learning Rate

In [ ]:
from tensorflow_addons.optimizers import CyclicalLearningRate
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

BATCH_SIZE = 50
EPOCHS = 4000
MIN_LR = 5e-5
MAX_LR = 1e-3
CYCLE_LENGTH = 200

steps_per_epoch = len(X_train) // BATCH_SIZE

# Define the learning rate cycle properties
clr = CyclicalLearningRate(
    initial_learning_rate=MIN_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size= CYCLE_LENGTH * steps_per_epoch
)

optimizer = Adam(clr)

# Display learning rate evolution
epochs = np.arange(0, EPOCHS)
lr = clr(steps_per_epoch * epochs)
plt.plot(epochs, lr)
plt.xlabel("Epochs")
plt.ylabel("Learning Rate")
plt.show()

### Custom callback

We define a custom callback that store the current best model weights.

In [16]:
from tensorflow.keras.callbacks import Callback

# custom Keras Callback that store 
class Auto_Save(Callback):
    best_weights = []
    def __init__(self):
        super(Auto_Save, self).__init__()
        self.best = np.Inf
                
    def on_epoch_end(self, epoch, logs=None):
        current_loss = logs.get('val_loss')
        if np.less(current_loss, self.best):
            self.best = current_loss            
            Auto_Save.best_weights = self.model.get_weights()
            
    def on_train_end(self, logs=None):
        if self.params['verbose'] == 2:
            print('\nSaved best {0:6.4f}\n'.format(self.best))

### Custom callback to display Learning rate during training

We define a custom callback to display the evolution of the learning rate

In [17]:

class Print_LR(Callback):    
    def on_epoch_end(self, epoch, logs=None):
        iteration = self.model.optimizer.iterations.numpy()
        lr = clr(iteration).numpy()
        if self.params['verbose'] == 2:
            print("Iteration {} - Learning rate: {}".format(iteration, lr) )

### Keras model definition

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, SpatialDropout1D,BatchNormalization,Flatten, Dropout, Input, MaxPool1D
from typing import Dict, Iterable, Any

def keras_model(meta: Dict[str, Any]):
    input_shape = meta["X_shape_"][1:]
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(SpatialDropout1D(0.2))
    model.add(Conv1D (filters=64, kernel_size=3, padding="same", activation='swish'))
    model.add(Conv1D (filters=64, kernel_size=3, padding="same", activation='swish'))
    model.add(MaxPool1D(pool_size=7,strides=5))
    model.add(SpatialDropout1D(0.2))
    model.add(Conv1D (filters=128, kernel_size=3, padding="same", activation='swish'))
    model.add(Conv1D (filters=128, kernel_size=3, padding="same", activation='swish'))
    model.add(MaxPool1D(pool_size=7,strides=5))
    model.add(SpatialDropout1D(0.2))
    model.add(Flatten())
    model.add(Dense(units=2048, activation="swish"))
    model.add(Dropout(0.2))
    model.add(Dense(units=2048, activation="swish"))
    model.add(Dense(units=1, activation="sigmoid"))
    # we compile the model with the custom Adam optimizer
    model.compile(loss = 'mean_squared_error', metrics=['mse'], optimizer = optimizer)
    # model.summary()
    return model

## Complex pipeline

In this pipeline we decompose transformers and estimators to be able to use custom validation data in the fitting process

In [15]:


from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

from pinard.nirs_pipelines import FeatureAugmentation

from scikeras.wrappers import KerasRegressor

from tensorflow.keras.callbacks import EarlyStopping

# Y scaler to scale test data set to create Y validation data
y_scaler = MinMaxScaler()
y_scaler.fit(y_train.reshape((-1,1)))
y_valid = y_scaler.transform(y_test.reshape((-1,1)))


# X transformation pipeline to create X validation data
transformer_pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('preprocessing', FeatureAugmentation(preprocessing)), 
])

## we could have used a full standard pipeline and access transformers with:
## estimator.regressor_[:-1].transform(X_test)
transformer_pipeline.fit(X_train)
X_valid = transformer_pipeline.transform(X_test)

# KerasRegressor definition, with validation data and callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=200, verbose=0, mode='min') 

k_regressor = KerasRegressor(model = keras_model,
                            callbacks=[Auto_Save(), Print_LR(), early_stop],
                            epochs=EPOCHS, 
                            fit__batch_size=BATCH_SIZE,
                            fit__validation_data = (X_valid, y_valid),
                            verbose = 2)

# estimation pipeline
pipeline = Pipeline([
    ('trans', transformer_pipeline), 
    ('KerasNN', k_regressor)
])

estimator = TransformedTargetRegressor(regressor = pipeline, transformer = y_scaler)
estimator.fit(X_train, y_train)

# We update the keras model to the saved weights (note the use of _ for regressor and model)
estimator.regressor_[1].model_.set_weights(Auto_Save.best_weights)

Y_preds = estimator.predict(X_test)
print("MAE", mean_absolute_error(y_test, Y_preds))
print("MSE", mean_squared_error(y_test, Y_preds))
print("MAPE", mean_absolute_percentage_error(y_test, Y_preds))
print("R²", r2_score(y_test, Y_preds))

TRANSFORM
ahahah (91, 2151, 9)
Epoch 1/4000
{'verbose': 2, 'epochs': 4000, 'steps': 6}
Iteration 8747 - Learning rate: 9.435311221750453e-05
6/6 - 3s - loss: 0.0346 - mse: 0.0346 - val_loss: 0.0374 - val_mse: 0.0374 - 3s/epoch - 479ms/step
Epoch 2/4000
{'verbose': 2, 'epochs': 4000, 'steps': 6}
Iteration 8753 - Learning rate: 9.470939403399825e-05
6/6 - 0s - loss: 0.0275 - mse: 0.0275 - val_loss: 0.0256 - val_mse: 0.0256 - 441ms/epoch - 74ms/step
Epoch 3/4000
{'verbose': 2, 'epochs': 4000, 'steps': 6}
Iteration 8759 - Learning rate: 9.506561764283106e-05
6/6 - 0s - loss: 0.0245 - mse: 0.0245 - val_loss: 0.0250 - val_mse: 0.0250 - 431ms/epoch - 72ms/step
Epoch 4/4000
{'verbose': 2, 'epochs': 4000, 'steps': 6}
Iteration 8765 - Learning rate: 9.542189945932478e-05
6/6 - 0s - loss: 0.0223 - mse: 0.0223 - val_loss: 0.0271 - val_mse: 0.0271 - 247ms/epoch - 41ms/step
Epoch 5/4000
{'verbose': 2, 'epochs': 4000, 'steps': 6}
Iteration 8771 - Learning rate: 9.577812306815758e-05
6/6 - 0s - loss: 

KeyboardInterrupt: 

## Cross validation with Keras Regressor

sklearn cv_validation does not consider the best version of the keras regressor.
The following example handles the problem.

In [18]:
from sklearn.model_selection import RepeatedKFold

kfold = RepeatedKFold(n_splits= 4, n_repeats= 2, random_state=rd_seed)

fold = 0
for train, test in kfold.split(y):
    X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
    
    # prepare validation data
    y_scaler = MinMaxScaler()
    y_scaler.fit(y_train.reshape((-1,1)))
    y_valid = y_scaler.transform(y_test.reshape((-1,1)))
    transformer_pipeline.fit(X_train)
    X_valid = transformer_pipeline.transform(X_test)
    
    # declare and fit regressor    
    k_regressor = KerasRegressor(model = keras_model,
                            callbacks=[Auto_Save(), Print_LR(), early_stop],
                            epochs=EPOCHS, 
                            fit__batch_size=BATCH_SIZE,
                            fit__validation_data = (X_valid, y_valid),
                            verbose = 0)

    pipeline = Pipeline([
        ('trans', transformer_pipeline), 
        ('KerasNN', k_regressor)
    ])

    estimator = TransformedTargetRegressor(regressor = pipeline, transformer = y_scaler)
    estimator.fit(X_train, y_train)
    
    # Predict test fold
    estimator.regressor_[1].model_.set_weights(Auto_Save.best_weights)
    Y_preds = estimator.predict(X_test)
    
    print("Fold:", fold, "- MSE:", mean_squared_error(y_test, Y_preds), "- R²:", r2_score(y_test, Y_preds))
    fold += 1

TRANSFORM
ahahah (91, 2151, 9)
TRANSFORM
ahahah (91, 2151, 9)
Fold: 0 - MSE: 1.6849190149106374 - R²: 0.823429833870391
TRANSFORM
ahahah (90, 2151, 9)
TRANSFORM
ahahah (90, 2151, 9)
Fold: 1 - MSE: 1.1147567555795974 - R²: 0.8819330510872447
TRANSFORM
ahahah (90, 2151, 9)
TRANSFORM
ahahah (90, 2151, 9)
Fold: 2 - MSE: 1.9768063395877722 - R²: 0.7478761490453487
TRANSFORM
ahahah (90, 2151, 9)
TRANSFORM
ahahah (90, 2151, 9)
Fold: 3 - MSE: 1.2772412090041212 - R²: 0.8237457752934785
TRANSFORM
ahahah (91, 2151, 9)
TRANSFORM
ahahah (91, 2151, 9)
Fold: 4 - MSE: 1.6152029402175572 - R²: 0.8583026152591565
TRANSFORM
ahahah (90, 2151, 9)
TRANSFORM
ahahah (90, 2151, 9)
Fold: 5 - MSE: 0.9099092350277839 - R²: 0.8510437431481825
TRANSFORM
ahahah (90, 2151, 9)
TRANSFORM
ahahah (90, 2151, 9)
Fold: 6 - MSE: 1.6856680199189804 - R²: 0.8045863507470962
TRANSFORM
ahahah (90, 2151, 9)


KeyboardInterrupt: 